In [16]:
import numpy as np
import pandas as pd

In [17]:
def find_differences_in_timestamp(df1, df2, name_timstamp1, name_timstamp2):
    if not name_timstamp1 in df1.columns:
        raise Exception("'timestamp' column not found in df1")
    if not name_timstamp2 in df2.columns:
        raise Exception("'timestamp' column not found in df2")

    if len(df1) != len(df2):
        print(len(df1))
        print(len(df2))
        raise Exception(
            f"df1 and df2 have different lengths ({len(df1)}, {len(df2)})")

    t1 = df1[name_timstamp1].values
    t2 = df2[name_timstamp2].values

    t1_sorted = np.sort(t1)
    t2_sorted = np.sort(t2)

    df = pd.DataFrame({
        "t1": t1_sorted,
        "t2": t2_sorted
    })

    df["diff"] = df["t1"] == df["t2"]

    return df

In [18]:
df1=pd.read_csv('data/processed/bpic15_time_manipulated.csv')
df2=pd.read_csv('bpic15_amended_typed.csv')

df1['time:timestamp'] = df['time:timestamp'].str.slice(stop=-5)

df1.head()
df2.head()


,Unnamed: 0.1,Unnamed: 0,Case ID,activity,Resource,Complete Timestamp,ct:last_phase,case_parts,Activity,r:municipality,ct:permit_type,at:phase,tt:month,tt:day,tt:weekday,tt:ampm,CT,AT,TT,CO
0,0,0,4284682,enter senddate procedure confirmation,560852,2009-11-18 09:00:00,Besluit genomen,"Bouw,Sloop",01_HOOFD_190_2,muni-4,Bouw,01_HOOFD_1,Nov,Day_18,Wed,AM,NaN,0-1-4-5,M-T-W-T-F,null/0-1-4-5/M-T-W-T-F
1,1,1,4511735,register submission date request,1254625,2009-11-23 09:00:00,Vergunning onherroepelijk,Brandveilig gebruik (vergunning),01_HOOFD_010,muni-5,Non Bouw,01_HOOFD_0,Nov,Day_23,Mon,AM,NaN,0-1-4-5,M-T-W-T-F,null/0-1-4-5/M-T-W-T-F
2,2,2,3090877,enter senddate procedure confirmation,3148844,2010-01-01 09:00:00,Vergunning verleend,Milieu (vergunning),01_HOOFD_065_2,muni-3,Non Bouw,01_HOOFD_0,Jan,Day_1,Fri,AM,NaN,0-1-4-5,M-T-W-T-F,null/0-1-4-5/M-T-W-T-F
3,3,4,3871304,date for inspection MER,560532,2010-06-29 08:00:00,Procedure afgebroken,Milieu (vergunning),01_HOOFD_080,muni-2,Non Bouw,01_HOOFD_0,Jun,Day_29,Tue,AM,NaN,0-1-4-5,M-T-W-T-F,null/0-1-4-5/M-T-W-T-F
4,4,6,3007101,register submission date request,560749,2010-10-04 08:00:00,Aanvraag ontvangen,Milieu (vergunning),01_HOOFD_010,muni-3,Non Bouw,01_HOOFD_0,Oct,Day_4,Mon,AM,NaN,0-1-4-5,M-T-W-T-F,null/0-1-4-5/M-T-W-T-F


In [22]:
diff = find_differences_in_timestamp(df1, df2, "time:timestamp", "Complete Timestamp")

diff

df1[~df1['time:timestamp'].str.endswith("00:00")]


,Unnamed: 0,case:concept:name,activity label,org:resource,time:timestamp,ct:last_phase,case_parts,concept:name,r:municipality,ct:permit_type,at:phase,tt:month,tt:day,tt:weekday,tt:ampm
